In [84]:
from tslearn.clustering import TimeSeriesKMeans
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
from urllib.request import urlopen
import json

In [85]:
df = pd.read_csv('Data_Files/price_by_state_cleaned.csv')
# print(df)

split_date = '2020-01-31'  # Choose the date where to split the data
train = df[df['Unnamed: 0'] < split_date]
train = train.drop(columns=['Unnamed: 0'])
test = df[df['Unnamed: 0'] >= split_date]
test = test.drop(columns=['Unnamed: 0'])

train = train.values.T
print(train.shape)

(51, 240)


In [86]:
km = TimeSeriesKMeans(n_clusters=6, max_iter=1000, random_state=0)
pred = km.fit_predict(train)
print(pred)

[2 3 3 5 3 3 1 3 3 1 4 0 5 0 4 3 1 5 1 3 5 3 3 1 3 1 5 1 0 0 1 0 1 1 1 3 1
 3 1 2 0 3 0 3 0 3 3 5 2 0 0]


In [98]:
state_names = df.columns.tolist()[1:]

plot_df = pd.DataFrame({
    "State": state_names,
    "Number": pred
})

cluster = {}

for i in range(6):
    state_with_ = plot_df[plot_df["Number"] == i]["State"]
    cluster[i] = state_with_.values.tolist()
print(cluster)

{0: ['Virginia', 'Arizona', 'Connecticut', 'Utah', 'Nevada', 'New Hampshire', 'Rhode Island', 'Delaware', 'Vermont', 'Wyoming'], 1: ['Ohio', 'Michigan', 'Indiana', 'Missouri', 'Alabama', 'Kentucky', 'Oklahoma', 'Iowa', 'Arkansas', 'Mississippi', 'Kansas', 'Nebraska', 'West Virginia'], 2: ['California', 'Hawaii', 'District of Columbia'], 3: ['Texas', 'Florida', 'Pennsylvania', 'Illinois', 'Georgia', 'North Carolina', 'Tennessee', 'Wisconsin', 'Minnesota', 'South Carolina', 'Louisiana', 'New Mexico', 'Idaho', 'Maine', 'Montana', 'South Dakota', 'North Dakota'], 4: ['New Jersey', 'Massachusetts'], 5: ['New York', 'Washington', 'Maryland', 'Colorado', 'Oregon', 'Alaska']}
